In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import quantstats as qs
import matplotlib.pyplot as plt
import seaborn as sns
import shutil 
import os
qs.extend_pandas()

import warnings
warnings.filterwarnings('ignore')

## Reading the trading signals data

In [2]:
df_trading_signals = pd.read_csv("data/trading_signals.csv", index_col = 0, parse_dates=True)

In [3]:
df_trading_signals.head()

,Price-PNC,Price-BAC,BAC-PNC,BAC-PNC-signals,Price-MS-PK,Price-USB-PH,MS-PK-USB-PH,MS-PK-USB-PH-signals,Price-CRZBY,CRZBY-BAC,CRZBY-BAC-signals
Date,,,,,,,,,,,
2022-08-11,170.253983,35.674946,-0.256736,0,25.390839,20.813704,-1.566423,1,7.08,-0.937327,0
2022-08-12,172.917603,36.062393,-0.445307,0,25.380985,20.863144,-1.708225,1,7.34,-0.673181,0
2022-08-15,172.224457,36.012718,-0.340061,0,25.440104,20.962021,-1.831263,1,7.21,-0.846369,0
2022-08-16,173.323578,36.400166,-0.166336,0,25.400692,20.952133,-1.886425,1,7.27,-0.871479,0
2022-08-17,171.640244,36.171673,-0.027179,0,25.144518,20.645611,-1.650691,1,7.14,-0.992813,0


## Creating a function for backtest

In [4]:
def backtest(df_trading, asset1 : str, asset2 : str):
    # Calculate profit and loss for each asset separately
    # Column names
    
    money = 10^5
    
    asset1_price = "Price-" + str(asset1)
    asset2_price = "Price-" + str(asset2)
    zscore = str(asset1) + "-" + str(asset2)
    signals = str(asset1) + "-" + str(asset2) + "-" + "signals"
    
    df = df_trading[[asset1_price, asset2_price, zscore, signals]].copy(deep = False)
    
    df["positions1"] = df[signals].diff()
    df["signals2"] = -df[signals]
    df["positions2"] = df["signals2"].diff()
    
    df['z upper limit'] = np.mean(df[zscore]) + np.std(df[zscore])
    df['z lower limit'] = np.mean(df[zscore]) - np.std(df[zscore])
    
    
    
    # floor division of no.of shares 
    no_of_shares_1 = money // max(df[asset1_price]) #12 // 5 = 2
    no_of_shares_2 = money // max(df[asset2_price])
    
    # asset 1
    portfolio = pd.DataFrame()
    portfolio['asset1'] = df[asset1_price]
    portfolio['holdings1'] = df['positions1'].cumsum() * df[asset1_price] * no_of_shares_1 
    portfolio['remaining_cash'] = money - (df['positions1'] * df[asset1_price] * no_of_shares_1).cumsum() 
    portfolio['total asset1'] = portfolio['holdings1'] + portfolio['remaining_cash'] 
    portfolio['return1'] = portfolio['total asset1'].pct_change()

    
    # asset 2
    portfolio['asset2'] = df[asset2_price]
    portfolio['holdings2'] = df['positions2'].cumsum() * df[asset2_price] * no_of_shares_2
    portfolio['remaining_cash2'] = money - (df['positions2'] * df[asset2_price] * no_of_shares_2).cumsum()
    portfolio['total asset2'] = portfolio['holdings2'] + portfolio['remaining_cash2']
    portfolio['return2'] = portfolio['total asset2'].pct_change()

    # combining both profit and loss
    portfolio['z'] = df[zscore]
    portfolio['total asset'] = portfolio['total asset1'] + portfolio['total asset2']
    portfolio['z upper limit'] = df['z upper limit']
    portfolio['z lower limit'] = df['z lower limit']
    portfolio.dropna(inplace = True)

    # calculate CAGR (compounded annual growth rate)
    final_portfolio = portfolio['total asset'].iloc[-1] # last value in the column total asset
    delta = (portfolio.index[-1] - portfolio.index[0]).days
    print('Number of days = ', delta)
    YEAR_DAYS = 252
    returns = (final_portfolio/money) ** (delta/YEAR_DAYS) - 1
    print('CAGR = {:.3f}%' .format(returns * 100))
    
    return df, portfolio

In [5]:
df1, portfolio1 = backtest(df_trading=df_trading_signals, asset1 = "BAC", asset2 ="PNC")

Number of days =  67
CAGR = 20.236%


In [6]:
df2, portfolio2 = backtest(df_trading=df_trading_signals, asset1 = "MS-PK", asset2 ="USB-PH")

Number of days =  67
CAGR = 20.236%


In [7]:
df3, portfolio3 =  backtest(df_trading=df_trading_signals, asset1 = "CRZBY", asset2 ="BAC")

Number of days =  67
CAGR = 21.018%
